In [11]:
from fastlite import *
from dataclasses import dataclass
from fastcore.utils import *
from typing import Optional

### Create table using parameters and then create a dataclass from that 

In [7]:
db = database(":memory:")

In [8]:
# @dataclass
class User:
    id: int
    username: str
    email: str
    password: str
    role: str = "user"
    created_at: str = ""
    last_login: str = ""
    active: bool = True

In [9]:
user = db.create(User, pk="id")

In [16]:
print(user.schema)

CREATE TABLE [user] (
   [id] INTEGER PRIMARY KEY,
   [username] TEXT,
   [email] TEXT,
   [password] TEXT,
   [role] TEXT,
   [created_at] TEXT,
   [last_login] TEXT,
   [active] INTEGER
)


In [17]:
# Add users
user.insert(User(username="user1", email="u1@mail.com", role="user", password="abc"))

User(id=1, username='user1', email='u1@mail.com', password='abc', role='user', created_at='', last_login='', active=1)

In [18]:
user.insert(User(username="user2", email="u2@mail.com", role="user", password="def"))

User(id=2, username='user2', email='u2@mail.com', password='def', role='user', created_at='', last_login='', active=1)

In [21]:
# Test retrieving a user

In [22]:
query = "SELECT user.id FROM user where user.username='user1'"

In [23]:
db.q(query)

[{'id': 1}]

In [24]:
query = "SELECT * FROM user where user.username='user1'"

In [25]:
db.q(query)

[{'id': 1,
  'username': 'user1',
  'email': 'u1@mail.com',
  'password': 'abc',
  'role': 'user',
  'created_at': '',
  'last_login': '',
  'active': 1}]

In [14]:
test_class = user.dataclass()
test_class

fastlite.core.User

In [20]:
hl_md(user.schema, 'sql')

```sql
CREATE TABLE [user] (
   [id] INTEGER PRIMARY KEY,
   [username] TEXT,
   [email] TEXT,
   [password] TEXT,
   [role] TEXT,
   [created_at] TEXT,
   [last_login] TEXT,
   [active] INTEGER
)
```

In [26]:
# Make username a primary key to simplify obtaining users

In [27]:
# @dataclass
class User2:
    id: int
    username: str
    email: str
    password: str
    role: str = "user"
    created_at: str = ""
    last_login: str = ""
    active: bool = True

In [39]:
user2 = db.create(User2, pk=("id", "username"))

In [40]:
hl_md(user2.schema, "sql")


```sql
CREATE TABLE [user2] (
   [id] INTEGER,
   [username] TEXT,
   [email] TEXT,
   [password] TEXT,
   [role] TEXT,
   [created_at] TEXT,
   [last_login] TEXT,
   [active] INTEGER,
   PRIMARY KEY ([id], [username])
)
```

In [41]:
user2.insert(User2(username="user1", email="u1@mail.com", role="user", password="abc"))
user2.insert(User2(username="user2", email="u2@mail.com", role="user", password="def"))

User2(id=None, username='user2', email='u2@mail.com', password='def', role='user', created_at='', last_login='', active=1)

In [43]:
user2("username=?", ('user1',))


[User2(id=None, username='user1', email='u1@mail.com', password='abc', role='user', created_at='', last_login='', active=1)]

In [81]:
def get_by_username(username: str) -> Optional[User]:
    try:
        user = user2("username=?", (username,))
        if len(user) == 1:
            return user[0]  # Return the single user object
        elif len(user) == 0:
            return None
        else:
            raise Exception(f"Multiple users found: {len(user)}")
    except Exception as e:
        print(f"Error in get_by_username: {e}")
        return None

In [82]:
get_by_username("user1")

User2(id=None, username='user1', email='u1@mail.com', password='abc', role='user', created_at='', last_login='', active=1)

The good thing about this approach is that it returns a dataclass object and not a dictionary

In [83]:
# Now add in a class method to update the last_login field

In [84]:
from datetime import datetime

In [87]:
# @dataclass
class User3:
    id: int
    username: str
    email: str
    password: str
    role: str = "user"
    created_at: str = ""
    last_login: str = ""
    active: bool = True
    pk="id"

    def __post_init__(self):
        now = datetime.now().isoformat()
        if not self.created_at:
            self.created_at = now
        if not self.last_login:
            self.last_login = now

In [89]:
user3 = db.create(User3, pk=User3.pk)

In [ ]:
def update(table, user_id: int, **kwargs) -> bool:
    """Update user fields"""
    try:
        table.update(kwargs, user_id)
        return True
    except Exception as e:
        print(f"Error updating user {user_id}: {e}")
        return False

In [109]:
test_user = User3(username="user3", email="u3@mail.com", role="user", password="abcd")

In [110]:
print(test_user)

User3(id=UNSET, username='user3', email='u3@mail.com', password='abcd', role='user', created_at='2025-09-01T15:43:16.181608', last_login='2025-09-01T15:43:16.181608', active=True)


In [111]:
user3.insert(test_user)

User3(id=3, username='user3', email='u3@mail.com', password='abcd', role='user', created_at='2025-09-01T15:43:16.181608', last_login='2025-09-01T15:43:16.181608', active=1)

In [90]:
user3.insert(User3(username="user1", email="u1@mail.com", role="user", password="abc"))
user3.insert(User3(username="user2", email="u2@mail.com", role="user", password="def"))

User3(id=2, username='user2', email='u2@mail.com', password='def', role='user', created_at='2025-08-29T17:43:05.657593', last_login='2025-08-29T17:43:05.657593', active=1)

In [102]:
test_user = user3("username=?", ('user1',))[0]
test_user

User3(id=1, username='user1', email='u1@mail.com', password='abc', role='user', created_at='2025-08-29T17:43:05.656197', last_login='2025-08-29T17:43:05.656197', active=1)

In [105]:
now = datetime.now().isoformat()

In [104]:
user3.update({"last_login": now, "id": test_user.id})

User3(id=1, username='user1', email='u1@mail.com', password='abc', role='user', created_at='2025-08-29T17:43:05.656197', last_login='2025-08-29T17:49:06.046949', active=1)

In [107]:
user3.update(last_login=now,  id=test_user.id)

User3(id=1, username='user1', email='u1@mail.com', password='abc', role='user', created_at='2025-08-29T17:43:05.656197', last_login='2025-08-29T17:52:22.658774', active=1)